MODEL-1) Weather Prediciton Using Linear Rgression 

In [1]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
from sklearn.linear_model import LinearRegression

# Function to fetch weather data from Visual Crossing Weather API
def fetch_weather_data(api_url):
    response = requests.get(api_url)
    data = response.text
    # Convert CSV data into DataFrame
    df = pd.read_csv(StringIO(data))
    return df

# Function to perform linear regression analysis
def perform_linear_regression(df, feature_column, target_column):
    X = df[[feature_column]]
    y = df[target_column]
    model = LinearRegression()
    model.fit(X, y)
    return model

# Visual Crossing Weather API URL
api_url = 'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Bangalore?unitGroup=metric&include=days&key=89PEUKKUCC4RXQ3TBGUYZF2KC&contentType=csv'

# Fetch weather data
weather_data = fetch_weather_data(api_url)

# Filter DataFrame to include specified columns and add conditions and description
weather_data = weather_data[['datetime', 'tempmax', 'tempmin', 'temp', 'humidity', 'conditions', 'description']]

# Print DataFrame head for inspection
print("DataFrame head:")
print(weather_data.head())

# Perform linear regression analysis
feature_column = 'humidity'  # Assuming humidity is the feature column
target_column = 'temp'  # Assuming temperature is the target column
model = perform_linear_regression(weather_data, feature_column, target_column)

# Display model coefficients
print("Intercept:", model.intercept_)
print("Coefficient:", model.coef_)


DataFrame head:
     datetime  tempmax  tempmin  temp  humidity              conditions  \
0  2024-05-11     35.3     21.0  28.2      59.6  Rain, Partially cloudy   
1  2024-05-12     35.6     25.0  29.7      48.2  Rain, Partially cloudy   
2  2024-05-13     35.6     24.9  29.2      49.1  Rain, Partially cloudy   
3  2024-05-14     35.2     24.2  29.2      45.5        Partially cloudy   
4  2024-05-15     33.0     24.0  28.7      49.7                Overcast   

                                         description  
0  Partly cloudy throughout the day with early mo...  
1  Partly cloudy throughout the day with early mo...  
2  Becoming cloudy in the afternoon with early mo...  
3                  Partly cloudy throughout the day.  
4                   Cloudy skies throughout the day.  
Intercept: 36.34866901714905
Coefficient: [-0.14724558]


MODEL-2) Weather Prediciton using RNN

In [2]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Function to fetch weather data from Visual Crossing Weather API
def fetch_weather_data(api_url):
    response = requests.get(api_url)
    data = response.text
    # Convert CSV data into DataFrame
    df = pd.read_csv(StringIO(data))
    return df

# Function to prepare data for RNN model
def prepare_data_for_rnn(df, feature_column, target_column, look_back=1):
    # Scale the data
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(df[[feature_column, target_column]])
    # Create input and output sequences
    X, y = [], []
    for i in range(len(df) - look_back):
        X.append(scaled_data[i:(i + look_back), :])
        y.append(scaled_data[i + look_back, 1])  # Assuming temperature is the target column

    X, y = np.array(X), np.array(y)
    
    return X, y, scaler

# Visual Crossing Weather API URL
api_url = 'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Bangalore?unitGroup=metric&include=days&key=89PEUKKUCC4RXQ3TBGUYZF2KC&contentType=csv'

# Fetch weather data
weather_data = fetch_weather_data(api_url)

# Prepare data for RNN model
feature_column = 'humidity'  # Assuming humidity is the feature column
target_column = 'temp'  # Assuming temperature is the target column
look_back = 1  # Define the look_back variable
X, y, scaler = prepare_data_for_rnn(weather_data, feature_column, target_column, look_back)

# Split data into train and test sets
train_size = int(len(X) * 0.7)
test_size = len(X) - train_size
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Reshape input data to be 3-dimensional (samples, time steps, features)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 2))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 2))

# Build RNN model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 2)))
model.add(LSTM(units=50))
model.add(Dense(units=1))

# Compile model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train model
model.fit(X_train, y_train, epochs=100, batch_size=32)

# Evaluate model
loss = model.evaluate(X_test, y_test)
print("Test Loss:", loss)

# Make predictions on test data
predicted_temps_scaled = model.predict(X_test)

# Reshape predictions to match the shape of the original data
predicted_temps_scaled = np.reshape(predicted_temps_scaled, (predicted_temps_scaled.shape[0], 1))

# Inverse transform the scaled predictions to get actual temperatures
predicted_temps = scaler.inverse_transform(np.concatenate((X_test[:, -1, 0].reshape(-1, 1), predicted_temps_scaled), axis=1))[:, 1]

# Create a DataFrame to store predictions and actual temperatures along with dates
predictions_df = pd.DataFrame({
    'Date': weather_data['datetime'].iloc[train_size + look_back:],
    'Actual_Temp': weather_data[target_column].iloc[train_size + look_back:],
    'Predicted_Temp': predicted_temps
})

# Display the results in tabular form
print(predictions_df)


Epoch 1/100


C:\Users\anant\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - loss: 0.4123
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - loss: 0.4050
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - loss: 0.3976
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - loss: 0.3904
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - loss: 0.3831
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - loss: 0.3758
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step - loss: 0.3686
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step - loss: 0.3613
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 0.3541
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 0.3468
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step - loss: 0.3394
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step - loss: 0.3320
Epoch 13/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step - loss: 0.3246
Epoch 14/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 0.3171
Epoch 15/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step - loss: 0.3096
Epoch 16/100
1/1 ━━━━━━━━━

MODLE-3) Weather Prediciton using CNN

In [3]:
import numpy as np
import requests
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Function to fetch weather data
def fetch_weather_data(api_url):
    response = requests.get(api_url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: Failed to fetch data from API. Status code: {response.status_code}")
        return None

# Weather API URL
api_url = 'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Bangalore?unitGroup=metric&key=89PEUKKUCC4RXQ3TBGUYZF2KC&contentType=json'

# Fetch weather data
weather_data = fetch_weather_data(api_url)

if weather_data is not None:
    # Extract weather features for the next 10 days
    forecast = weather_data['days'][:10]

    # Process weather features
    features = []
    for day in forecast:
        features.append([
            day['tempmax'],
            day['tempmin'],
            day['precip'],
            day['humidity'],
            day['windspeed']
        ])

    X = np.array(features)

    # Define and train a simple neural network model
    model = Sequential([
        Dense(64, activation='relu', input_shape=(X.shape[1],)),
        Dense(32, activation='relu'),
        Dense(3, activation='softmax')  # Output layer with 3 classes: humid, rainy, sunny
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the model with all available data
    model.fit(X, np.zeros(X.shape[0]), epochs=10, batch_size=32)  # No labels required for weather prediction

    # Make predictions for the next 10 days
    predictions = model.predict(X)

    # Interpret predictions and print forecast
    weather_classes = ['Humid', 'Rainy', 'Sunny']
    print("Weather Forecast for the next 10 days:")
    for i, day in enumerate(forecast):
        pred_class = weather_classes[np.argmax(predictions[i])]
        print(f"Day {i+1}: {pred_class} - Temperature: {day['tempmax']}°C (max), {day['tempmin']}°C (min), Precipitation: {day['precip']} mm, Humidity: {day['humidity']}%, Wind Speed: {day['windspeed']} km/h")


Epoch 1/10


C:\Users\anant\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0000e+00 - loss: 11.6509
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - accuracy: 0.0000e+00 - loss: 9.8236
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.0000e+00 - loss: 8.0207
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.0000e+00 - loss: 6.3471
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.0000e+00 - loss: 4.9069
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.0000e+00 - loss: 3.6055
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - accuracy: 0.0000e+00 - loss: 2.3858
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.0000e+00 - loss: 1.3438
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 0.9000 - loss: 0.6276
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step - accuracy: 1.0000 - loss: 0.2600
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
Weather Forecast for the next 10 days:
Day 1: Humid - Temperature: 35.3°C (max), 21.0°C (min), Precipitation: 

MODEL-4) Weather Predicition using ANN

In [4]:
import numpy as np
import requests
from tabulate import tabulate
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPClassifier

# Function to fetch weather data
def fetch_weather_data(api_url):
    response = requests.get(api_url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: Failed to fetch data from API. Status code: {response.status_code}")
        return None

# Weather API URL
api_url = 'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Bangalore?unitGroup=metric&key=89PEUKKUCC4RXQ3TBGUYZF2KC&contentType=json'

# Fetch weather data
weather_data = fetch_weather_data(api_url)

if weather_data is not None:
    # Extract weather features for the next 10 days
    forecast = weather_data['days'][:10]

    # Process weather features
    features = []
    targets = []
    weather_classes = ['Humid', 'Rainy', 'Sunny']
    for day in forecast:
        features.append([
            day['tempmax'],
            day['tempmin'],
            day['precip'],
            day['humidity'],
            day['windspeed']
        ])
        # Dummy targets for demonstration
        targets.append(np.random.randint(0, 3))

    X = np.array(features)
    y = np.array(targets)

    # Normalize feature data
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)

    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

    # Define and train the ANN model
    model = MLPClassifier(hidden_layer_sizes=(64, 32), activation='relu', max_iter=1000)
    model.fit(X_train, y_train)

    # Evaluate the model
    accuracy = model.score(X_test, y_test)
    print("Test Accuracy:", accuracy)

    # Make predictions for the next 10 days
    predictions = model.predict(X_scaled)

    # Interpret predictions and print forecast
    table_data = []
    for i, day in enumerate(forecast):
        pred_class = weather_classes[predictions[i]]
        table_data.append([f"Day {i+1}", day['datetime'], day['tempmax'], day['tempmin'], day['precip'], day['humidity'], day['windspeed'], pred_class])

    # Create table headers
    headers = ["Day", "Date", "Max Temp (°C)", "Min Temp (°C)", "Precipitation (mm)", "Humidity (%)", "Wind Speed (km/h)", "Predicted Weather"]

    # Print table
    print(tabulate(table_data, headers=headers, tablefmt="grid"))


Test Accuracy: 0.0
+--------+------------+-----------------+-----------------+----------------------+----------------+---------------------+---------------------+
| Day    | Date       |   Max Temp (°C) |   Min Temp (°C) |   Precipitation (mm) |   Humidity (%) |   Wind Speed (km/h) | Predicted Weather   |
+========+============+=================+=================+======================+================+=====================+=====================+
| Day 1  | 2024-05-11 |            35.3 |            21   |                  0.1 |           59.6 |                23.8 | Humid               |
+--------+------------+-----------------+-----------------+----------------------+----------------+---------------------+---------------------+
| Day 2  | 2024-05-12 |            35.6 |            25   |                  1.3 |           48.2 |                17.3 | Sunny               |
+--------+------------+-----------------+-----------------+----------------------+----------------+------------------

Comparison:

       Date  Actual_Temp  LR_Predictions  LSTM_Predictions  NN_Predictions  MLP_Predictions
0  5/11/2024           28       24.725068         26.074774       26.665476                2
1  5/12/2024           28       24.725068         26.074774       26.665476                2
2  5/13/2024           29       24.725068         26.074774       26.665476                2
3  5/14/2024           29       24.725068         26.074774       26.665476                2
4  5/15/2024           28       24.725068         26.074774       26.665476                2
5  5/16/2024           28       24.725068         26.074774       26.665476                2
6  5/17/2024           28       24.725068         26.074774       26.665476                2
7  5/18/2024           29       24.725068         26.074774       26.665476                2
8  5/19/2024           30       24.725068         26.074774       26.665476                2
9  5/20/2024           30       24.725068         26.074774       26.665476                2
